<a href="https://colab.research.google.com/github/AppimateSA/Latte/blob/luthando-contribution/experiments.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/AppimateSA/Latte.git
# %cd Latte
# !git checkout luthando-contribution

# Install Correct Modules

In [2]:
# !pip install git+https://github.com/huggingface/diffusers

# Import

In [3]:
import imageio
import torch
from torchvision.utils import save_image
from diffusers import LattePipeline
from diffusers.models import AutoencoderKLTemporalDecoder


torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

/home/luthando/miniconda3/envs/latte/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# !python -m pip uninstall diffusers -y && conda uninstall diffusers -y
# !conda clean -ay
# !python -m pip cache purge

# Normal Inference

In [5]:
# video_length = 16 # 1 (text-to-image) or 16 (text-to-video)
# pipe = LattePipeline.from_pretrained("maxin-cn/Latte-1", torch_dtype=torch.float16).to(device)

# # Using temporal decoder of VAE
# vae = AutoencoderKLTemporalDecoder.from_pretrained("maxin-cn/Latte-1", subfolder="vae_temporal_decoder", torch_dtype=torch.float16).to(device)
# pipe.vae = vae

In [6]:
# prompt = "a cat wearing sunglasses and working as a lifeguard at pool."
# videos = pipe(prompt, video_length=video_length, output_type='pt').frames.cpu()

# if video_length > 1:
#     videos = (videos.clamp(0, 1) * 255).to(dtype=torch.uint8) # convert to uint8
#     imageio.mimwrite('./latte_output.mp4', videos[0].permute(0, 2, 3, 1), fps=8, quality=5) # highest quality is 10, lowest is 0
# else:
#     save_image(videos[0], './latte_output.png')

# Inference with 4/8-bit quantization

In [7]:
import gc
from transformers import T5EncoderModel, BitsAndBytesConfig


torch.manual_seed(0)

def flush():
    gc.collect()
    torch.cuda.empty_cache()

def bytes_to_giga_bytes(bytes):
    return bytes / 1024 / 1024 / 1024

video_length = 16
model_id = "maxin-cn/Latte-1"

text_encoder = T5EncoderModel.from_pretrained(
    model_id,
    subfolder="text_encoder",
    quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16),
    device_map="auto",
)
pipe = LattePipeline.from_pretrained(
    model_id, 
    text_encoder=text_encoder,
    transformer=None,
    device_map="balanced",
)

Loading pipeline components...: 100%|██████████| 4/4 [00:00<00:00, 21.44it/s]


In [8]:
outputs_folder = "./outputs/"
with torch.no_grad():
    prompt = "a cat wearing sunglasses and working as a lifeguard at pool, with hot girls."
    negative_prompt = ""
    prompt_embeds, negative_prompt_embeds = pipe.encode_prompt(prompt, negative_prompt=negative_prompt)

del text_encoder
del pipe
flush()

pipe = LattePipeline.from_pretrained(
    model_id,
    text_encoder=None,
    torch_dtype=torch.float16,
).to("cuda")
# pipe.enable_vae_tiling()
# pipe.enable_vae_slicing()
videos = pipe(
    video_length=video_length,
    num_inference_steps=50,
    negative_prompt=None, 
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_prompt_embeds,
    output_type="pt",
).frames.cpu()
print(f"Max memory allocated: {bytes_to_giga_bytes(torch.cuda.max_memory_allocated())} GB")

Loading pipeline components...:   0%|          | 0/4 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The config attributes {'attention_type': 'default', 'double_self_attention': False, 'norm_num_groups': 32, 'num_vector_embeds': None, 'only_cross_attention': False, 'upcast_attention': False, 'use_linear_projection': False} were passed to LatteTransformer3DModel, but are not expected and will be ignored. Please verify your config.json configuration file.
Some weights of the model checkpoint were not used when initializing LatteTransformer3DModel: 
 ['caption_projection.y_embedding']
100%|██████████| 50/50 [01:01<00:00,  1.24s/it]


Max memory allocated: 9.150381565093994 GB


In [9]:
if video_length > 1:
    videos_uint8 = (videos.clamp(0, 1) * 255).to(dtype=torch.uint8) # convert to uint8
    imageio.mimwrite(f"{outputs_folder}latte_output_3.mp4", videos_uint8[0].permute(0, 2, 3, 1), fps=8, quality=5) # highest quality is 10, lowest is 0
else:
    save_image(videos[0], f"{outputs_folder}latte_output_3.png")